In [ ]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import *
spark=SparkSession.builder.master("local").appName("unified_to_refinemant").getOrCreate()

In [ ]:
#script execution for raw data to landing
class unified_to_refinement:
    
    """Script to read parquet file from local path to unified.."""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter
        
        """read parquet data from local"""
    def read_csv(self,fileformat,file_path):
        
        return spark.read.parquet(file_path)

        """save file as parquet and also save as table in local db"""
    def write_parquet(self,df,dest_path,table_name):   
        print(dest_path,table_name)
        df.write.mode("overwrite").parquet(dest_path)
        df.coalesce(1).write.format(self.file_parameter['parquet']).mode("overwrite").option("path",dest_path)\
        .saveAsTable("school_refinement.{0}".format(table_name))
        
        
        """creating temporary view to query the 1st rank students from school unified_data"""
        
    def create_view(self,df):
        df.createOrReplaceTempView(self.file_parameter['temp_table'])
        temp_df = spark.sql("select * from {0}".format(self.file_parameter['temp_table']))
        temp_df.show(truncate = False)
        highest_marks_df = spark.sql("select * from (select *, dense_rank() over(partition by subject_name, month(exam_date) order by marks DESC)as highest_mark  from Refinement_student_details)A where A.highest_mark = 1 order by month(exam_date)")
        return temp_df
    
    def main_block(self):
        
        """main execution block"""
        spark.sql("create database if not exists {0}".format(self.file_parameter['local_db_name']))
        students_details=self.read_csv(self.file_parameter['parquet_format'],self.file_parameter['src_base_read_path']+self.file_parameter['read_filename_school_unified']+self.file_parameter['parquet_format']);
       
        parquet_df1 = self.create_view(students_details);
        self.write_parquet(parquet_df1,self.file_parameter['dest_base_write_path']+self.file_parameter['write_merged_df'],self.file_parameter['table_name_school_refinement']);

        
        

In [ ]:
if __name__=='__main__':
    
    file_read={
        "csv_format":".csv",
        "csv":"csv",
        "parquet_format":".parquet",
        "parquet":"parquet",
        "table_name_school_unified":"unified_student_details",
        "table_name_students":"students",
        "table_name_subject":"subjects",
        "read_filename_marks":"/marks/markdetails",
        "read_filename_student":"/student/studentdetails",
        "read_filename_subject":"/subject/subjectdetails",
        
        "write_merged_df":"/school_unified/students_details.parquet",
        "dest_base_write_path":"/Users/vasanth_ku/Vasanth/spark-apache/git projects/big_data_project_git/data_files/unified",
        "src_base_read_path":"/Users/vasanth_ku/Vasanth/spark-apache/git projects/big_data_project_git/data_files/landing/school_landing",
        "temp_table":"school_table",
        "local_db_name":"school_unified",

    }
   
    

    unified_obj1=parquet_to_unified(file_read)
    unified_obj1.main_block()
   